In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Load the data
df = pd.read_csv('C:/Users/parak/Desktop/vacancyy/Vacancy_Model_flask/hotel_booking.csv')

In [ ]:
df

In [ ]:


# Create num_rooms_available column
df['num_rooms_available'] = df['reserved_room_type'].apply(lambda x: ord('G') - ord(x) + 1)

# Create num_rooms_booked column
df['num_rooms_booked'] = df['assigned_room_type'].apply(lambda x: ord('G') - ord(x) + 1)

# Create num_cancellations column
df['num_cancellations'] = df['previous_cancellations'] + df['previous_bookings_not_canceled']

# Create avg_length_of_stay column
df['avg_length_of_stay'] = df['stays_in_weekend_nights'] + df['stays_in_week_nights']

# Create location column
df['location'] = df['hotel'].apply(lambda x: 'City Hotel' if x == 'City Hotel' else 'Resort Hotel')


In [ ]:
# Prepare the data
X = df[['num_rooms_available', 'num_rooms_booked', 'num_cancellations', 'avg_length_of_stay', 'location']]
X = pd.get_dummies(X)
# y = df['vacancy_rate']

In [ ]:
df['vacancy_rate'] = (df['num_rooms_available'] - df['num_rooms_booked']) / df['num_rooms_available']


In [ ]:
y = df['vacancy_rate']

In [ ]:
df.vacancy_rate.head(20)

In [ ]:
df['vacancy_rate'].fillna(0, inplace=True)

In [ ]:
df = df[~df['vacancy_rate'].isin([np.nan, np.inf, -np.inf])]
df['vacancy_rate'] = df['vacancy_rate'].replace([np.nan, np.inf, -np.inf], df['vacancy_rate'].median())


In [ ]:
from sklearn.preprocessing import MinMaxScaler

y = df['vacancy_rate'].to_numpy().reshape(-1, 1)
scaler = MinMaxScaler()
y = scaler.fit_transform(y).flatten()


y = df['vacancy_rate'].to_numpy()
scaler = MinMaxScaler()
y = scaler.fit_transform(y.reshape(-1, 1)).flatten()


In [ ]:
# drop the rows in df that are not present in X
df = df[df.index.isin(X.index)]

# drop the rows in X that are not present in df
X = X[X.index.isin(df.index)]

# ensure that both datasets have the same number of samples
assert len(df) == len(X)


In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# y_train.replace(np.NaN,0.0)

# y_train2 = y_train.replace(np.NaN,0.0)

# y_test2 = y_test.replace(np.NaN,0.0)

import numpy as np

# assume y_train and y_test are NumPy ndarray objects
y_train2 = np.where(np.isnan(y_train), 0.0, y_train)
y_test2 = np.where(np.isnan(y_test), 0.0, y_test)





In [ ]:
# Train the model
# model = LinearRegression()
# model.fit(X_train, y_train)
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train2)

In [ ]:
# Evaluate the model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test2, y_pred)
print('Mean Squared Error:', mse)

In [ ]:
X_train.head()

In [ ]:
# Make predictions
new_data = pd.DataFrame({'num_rooms_available': [50], 'num_rooms_booked': [30], 'num_cancellations': [5], 'avg_length_of_stay': [3], 'location_City Hotel': [1],'location_Resort Hotel':[1]})
new_data = pd.get_dummies(new_data)
prediction = model.predict(new_data)
print('Predicted Vacancy Rate:', prediction)

In [ ]:
# to make pickle file
import pickle

# save the model as a pickle file
with open('vacancy_flask.pkl', 'wb') as f:
    pickle.dump(model, f)

